In [2]:
# Import libraries used 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Convert json file to a csv file

business_json_path = 'Data/yelp_business.json'

business_df = pd.read_json(business_json_path, lines=True)
business_df

In [4]:
#### General Cleaning#####
# Initial shape of data
print("Shape of data initially " + str(business_df.shape))

# Number of null values
print("This is the total number of null values in the dataframe " + str(business_df.isna().sum().sum()))

# Drop rows where restaraunt is not open
business_df = business_df[business_df.is_open != 0]

# Drop column 'is_open' as not needed anymore
business_df = business_df.drop(['is_open'], axis=1)

# Drop rows that have that do not have proper star rating and have "yes"
business_df = business_df[business_df['stars'] != 'Yes']
# Filter catagories that contain "Restaurants" and #Food
business_df = business_df[business_df['categories'].str.contains('Restaurants|Food', case =False, na=False)]



print("New shape of data " + str(business_df.shape))

In [5]:
# Create column that counts the number of null values in each row
business_df["Null Count"] = business_df.isnull().sum(axis=1)
business_df

In [6]:
# Separate the days of the week open from the hours column
business_df = pd.concat([business_df, business_df["hours"].apply(pd.Series)], axis=1)
business_df

In [7]:
# Replace all NaN values in all the day columns to have 0 to represent not being open that day 
business_df[['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday','Sunday']] = business_df[['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday','Sunday']].replace(np.nan,0)

In [8]:
# Function created to check if restaraunt is open on the weekend
def open_weekend_conditions(x):
    if (x['Saturday'] != 0) & (x['Sunday'] != 0):
        return 1
    else:
        return 0
# Apply Function to new column open_weekend in the dataframe
business_df["open_weekends"] = business_df.apply(open_weekend_conditions,axis=1)
business_df

In [9]:
# Is restaruant open on a Monday?
#Replace all null values in Monday to be 0 
business_df['Monday'] = business_df['Monday'].replace(np.nan,0)
# change 0:0 -0:0 to 0s 
#Function to convert monday 0 or 0:0-0:0 values to 0 and the rest be 1 to represent open.
business_df['Monday'] = business_df['Monday'].replace('0:0-0:0',0,regex=True)

#Function to convert open to 1 and 0 for restaraunts that are not open on monday
def open_on_monday(x):
    if (x['Monday'] != 0):
        return 1
    else:
        return 0

#Apply the function to create open_Monday column
business_df["open_monday"] = business_df.apply(open_on_monday,axis=1)

## Drop days of the week columns not needed anymore 
business_df = business_df.drop(business_df.iloc[:,14:21],axis=1)

## Drop hours column as not needed anymore
business_df = business_df.drop('hours', axis=1)

In [10]:
## Extract the attributes column to its own column
business_df = pd.concat([business_df, business_df["attributes"].apply(pd.Series)], axis=1)
business_df.head()
## Make all attributes columns that are nan to 0 
business_df= business_df.fillna(0)

# Make the 'None' type values into 0 throughout the dataframe

# Replace String using apply() function with lambda.
business_df = business_df.apply(lambda x: x.replace({"none": 0, "None": 0,"False": 0,"u'none'": 0," u 'no'": 0,'no': 0,}, regex=True))
business_df.head()

In [11]:
# Columns that can be taken out of dataFrame AgesAllowed HairSpecializesIn AcceptsInsurance BYOBCorkage GoodForDancing BestNights BYOB ByAppointmentOnly BusinessParking RestaurantsAttire Ambience CoatCheck GoodForMeal BusinessAcceptsBitcoin 
business_df = business_df.drop(['AgesAllowed','DietaryRestrictions','HairSpecializesIn', 'Music','AcceptsInsurance','Open24Hours','BYOBCorkage', 'Corkage','GoodForDancing','BestNights','BYOB','ByAppointmentOnly','BusinessParking','RestaurantsAttire','Ambience', 'CoatCheck', 'GoodForMeal', 'BusinessAcceptsBitcoin','RestaurantsCounterService'], axis = 1)

#Clean Wifi column
# Function to change having wifi to 1 and rest to 0
def clean_wifi(x):
    if (x['WiFi'] != 0):
        return 1
    else:
        return 0
business_df["WiFi"] = business_df.apply(clean_wifi,axis=1)

# Drop the attribiutes colums as all data is extracted from it 
business_df = business_df.drop(columns= 'attributes')


### clean the u before the string

In [12]:
# Remove the starting u', ', and the ending ' for NoiseLevel
business_df['NoiseLevel'] = business_df['NoiseLevel'].str.replace("^u", "")
business_df['NoiseLevel'] = business_df['NoiseLevel'].str.replace("\'", "")
business_df['NoiseLevel'].value_counts()

In [13]:
# Remove the starting u', ', and the ending ' for Smoking
business_df['Smoking'] = business_df['Smoking'].str.replace("^u", "")
business_df['Smoking'] = business_df['Smoking'].str.replace("\'", "")
business_df['Smoking'].value_counts()

In [14]:
# Remove the starting u', ', and the ending ' for Alcohol
business_df['Alcohol'] = business_df['Alcohol'].str.replace("^u", "")
business_df['Alcohol'] = business_df['Alcohol'].str.replace("\'", "")
business_df['Alcohol'].value_counts()

In [15]:
# Simplify to- Change all "False" values to 0 and "True" values to 1       
business_df = business_df.replace({'False': '0', 'True': '1'})

In [16]:
## ordinal value converison work
###Make Noise Level column into ordinal values###
# change the null values to 0
business_df["NoiseLevel"] = business_df["NoiseLevel"].replace(np.nan,0)
# Encode NoiseLevel values in column
scale_mapper = {"quiet":1, "average":2, "loud":3, "very_loud":4}
business_df["NoiseLevel"] = business_df["NoiseLevel"].replace(scale_mapper)

## Make Alcohol column into ordinal values
business_df["Alcohol"] = business_df["Alcohol"].replace(np.nan,0)
alcohol_mapper = {"full_bar":1, "beer_and_wine":1}
business_df["Alcohol"] = business_df["Alcohol"].replace(alcohol_mapper)

## Clean smoking column into ordinal values
business_df["Smoking"] = business_df["Smoking"].replace(np.nan,0)
smoking_mapper = {"outdoor":1, "yes":1}
business_df["Smoking"] = business_df["Smoking"].replace(smoking_mapper)

In [17]:
#pd.get_dummies(business_df)
#['city','state','stars','review_count' 'RestaurantsDelivery','OutdoorSeating','BusinessAcceptsCreditCards','BikeParking','RestaurantsTakeOut','WiFi', 'Alcohol','Caters', 'WheelchairAccessible', 'GoodForKids','RestaurantsReservations', 'DogsAllowed','RestaurantsTableService', 'RestaurantsGoodForGroups', 'HasTV','HappyHour','DriveThru','Smoking']

In [18]:
# Export Cleaned Data
business_df.to_csv('Data/yelp_business_cleaned.csv',
                    index = False)



In [21]:
import os
os.listdir('Data')

### Create ordinal/dummy variables of your columns:

Best Code Source for dummy variables: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

pd.dummy()

Overview of dummy variable concept: https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/

Split out the columns of attributes and categories into dummy variables

### Exploratory Data Analysis

In [22]:
# Load data
business_df = pd.read_csv('Data/yelp_business_cleaned.csv')

In [23]:
business_df.head()

In [ ]:
# Create dummy variables for ethnic foods

business_df['Vietnamese'] = business_df['categories'].str.contains("|".join(["Vietnamese","Vietnam"]))

business_df['Korean'] = business_df['categories'].str.contains("|".join(["Korean","Korea"]))

business_df["Chinese"] = business_df['categories'].str.contains("Chinese","China")
business_df["Italian"] = business_df['categories'].str.contains("|".join(["Italian","Italy","Pasta"]))
business_df["Mexican"] = business_df['categories'].str.contains("|".join(["Mexican","Mexico","Buritos","Tacos"]))
business_df["Indian"] = business_df['categories'].str.contains("Indian","Curry")

# Create a column that states the ethnicity of the food for EDA
# Create ethnic food column
business_df['ethnic_food'] = ""
business_df["ethnic_food"][business_df['Vietnamese'] == 1] = "Vietnamese"
business_df["ethnic_food"][business_df['Korean'] == 1] = "Korean"
business_df["ethnic_food"][business_df['Chinese'] == 1] = "Chinese"
business_df["ethnic_food"][business_df['Italian'] == 1] = "Italian"
business_df["ethnic_food"][business_df['Mexican'] == 1] = "Mexican"
business_df["ethnic_food"][business_df['Indian'] == 1] = "Indian"
business_df["ethnic_food"].value_counts()

# Filter to only ethnic foods

ethnic_foods_df = business_df[business_df["ethnic_food"] != ""]


# change the 0-1s to yes and no for exploratory data analysis
ethnic_foods_df = ethnic_foods_df.replace({0: 'No', 1: 'Yes', '0': 'No', '1': 'Yes'})
ethnic_foods_df = ethnic_foods_df[ethnic_foods_df['stars'] != 'Yes']


In [ ]:
# Bar plot of the main ethnic food categories
plt.figure(figsize=(8,5))
sns.countplot(x = ethnic_foods_df["ethnic_food"])
plt.title("Ethnic Foods in the Data")

In [ ]:
# Average star reviews by ethnicity
ethnic_foods_df['stars'] = ethnic_foods_df['stars'].astype(int)
five_star_review = ethnic_foods_df.groupby('ethnic_food')['stars'].mean()
five_star_review = five_star_review.to_frame().reset_index()
five_star_review.plot(kind='barh',x='ethnic_food', title='Average Star Rating per Ethnic Food')

In [ ]:
ethnic_foods_df.head()

In [ ]:
ethnic_foods_df['stars'].value_counts()

In [ ]:
# Most 5 star reviews by ethnicity
five_star_review = ethnic_foods_df.query("stars == 5").groupby('ethnic_food')['stars'].count()
five_star_review = five_star_review.to_frame().reset_index()
five_star_review.plot(kind='barh',x='ethnic_food', title='Ethnic Foods with most 5 star reviews from users')

In [ ]:
# Visualizations to see how ethnic foods are offering restaraunt delivery
plt.figure(figsize=(8,5))
sns.countplot(data=ethnic_foods_df, x="ethnic_food", hue="RestaurantsDelivery")
plt.title("Count of Ethnic Foods and Delivery Capability")

In [ ]:
# State and ethnic foods
plt.figure(figsize=(8,5))
sns.countplot(x='state',data=ethnic_foods_df, palette='Set2',hue='ethnic_food')
plt.title("Count of Ethnic Foods within Each State")

In [ ]:
#State and open monday analysis
# Review Count vs star rating Scatterplot
# Group by manipulation for state and open monday

state_open_monday_analysis_df = business_df.groupby(['state','open_monday']).agg(mean_star_rating=("stars", 'mean'))
state_open_monday_analysis_df = state_open_monday_analysis_df.reset_index()
plt.figure(figsize=(8,5))
ax = sns.barplot(x="state",
           y="mean_star_rating",
           hue="open_monday",
           data= state_open_monday_analysis_df)
plt.title("Mean Star Rating and Open Monday Comparison")
legend_handles, _= ax.get_legend_handles_labels()
ax.legend(legend_handles, ['No','Yes'], 
          bbox_to_anchor=(1,1), 
          title='open_monday')

In [ ]:
#State and open_weekend analysis
# Review Count vs star rating Scatterplot
# Group by manipulation for state and open monday

state_open_monday_analysis_df = business_df.groupby(['state','open_weekends']).agg(mean_star_rating=("stars", 'mean'))
state_open_monday_analysis_df = state_open_monday_analysis_df.reset_index()
plt.figure(figsize=(8,5))
ax = sns.barplot(x="state",
           y="mean_star_rating",
           hue="open_weekends",
           data= state_open_monday_analysis_df)
plt.title("Mean Star Rating and Open Weekend Comparison")
legend_handles, _= ax.get_legend_handles_labels()
ax.legend(legend_handles, ['No','Yes'], 
          bbox_to_anchor=(1,1), 
          title='open_weekends')

In [ ]:
#Visualize what the star ratings look like and the price range
plt.figure(figsize=(8,5))
sns.displot(business_df, x="stars", hue='RestaurantsPriceRange2', multiple="stack", discrete=True)
plt.title("Distribution of Star Ratings and Price Range")

In [ ]:
# Number of reviews compared to price range


sns.catplot(data=business_df, x="RestaurantsPriceRange2", y="review_count")
plt.title("Relationship Between Price Range and Number of Reviews")

In [ ]:
# Cities with most Business Reviews
review_count_by_city = business_df.groupby('city')['review_count'].sum()
review_count_by_city.sort_values(ascending=False)[:10].plot(kind='barh', title='Top 10 Cities with the most Business Reviews')

In [ ]:
#Scatter plot matrix



In [ ]:
business_df.head()

In [ ]:
#dummies = pd.get_dummies(business_df["state"])
#pd.concat([business_df, dummies])

In [ ]:
business_df.head()

In [ ]:
dummies = business_df['categories'].str.get_dummies(',')
business_df = pd.concat([business_df, dummies])
business_df

In [ ]:
ethnic_food_category = ["American", "Vietnamese", "Korean", "Italian", "Mexican", "Japanese", "Chinese"]

In [ ]:
business_df['Vietnamese'].value_counts()

In [ ]:
business_df["ethnic food category"] = ""

business_df["B"][business_df['A'].str.contains("BULL")] = "Long"

In [ ]:
# EDA
# Compare rating by food category (American, Mexican, Indian, etc) by state

In [ ]:
n = int(input())
arr = list(map(int, input().split()))
print(arr)

In [ ]:
# use geek for geeks for looking up solutions

# noise level into ordinal values

# clean the categories - break out columns series.stri
 
#Different categories of food work for different states

In [ ]:
# city, state to dummy variable

# change the true false to 0 and 1


# Try 3 algorithms: Linear regression, Naive Bayes, XGBoost


# 5 fold cross validation

# https://machinelearningmastery.com/k-fold-cross-validation/



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fcd92ccc-2af6-49a1-bc78-ae9f0a350628' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>